# Sentiment Classification 

In [79]:
from os import getcwd, chdir
import re
import numpy as np
import pickle as pk
import pandas as pd

from nltk.metrics import ConfusionMatrix
from nltk.classify import NaiveBayesClassifier, MaxentClassifier
from nltk.classify import accuracy
from nltk.tokenize import word_tokenize as wt

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score

# Change your path here
fpath = getcwd()
print(fpath)

/Users/pierlim/PycharmProjects/sent_mining_CA


In [80]:
df = pd.read_csv('./data/df_reviews.csv')
df.head()

,rating,review,sentiment
0,5,The service at Sushi Bar was great with an ext...,1
1,2,OMG Yelp let me down big time. I don't know h...,-1
2,4,Heard that this was the best sushi joint in Si...,1
3,3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,5,Yes I also get the raving reviews now.\nWe dec...,1


## Use Standard Dataset to combine with scrapped data

In [110]:
ffile1 = open("./data/train.csv","r", encoding = "ISO-8859-1")
df_standard = pd.read_csv(ffile1, encoding = "utf-8")
df_standard.drop(['restaurant_id', 'date', 'review_id', 'stars'], inplace=True, axis=1)
df_standard.rename(columns={'text': 'review', 'Sentiment': 'sentiment'}, inplace=True)
df_standard.head()

,review,sentiment
0,Very disappointed in the customer service. We ...,negative
1,I really wasn't thrilled with our meal here. T...,negative
2,STAY AWAY...\n\nWe've been 3 times over the pa...,negative
3,The food is good and the portions are large. ...,negative
4,I feel bad about giving this place such a meh ...,negative


In [113]:
mask = df_standard.sentiment == 'negative'
column_name = 'sentiment'
df_standard.loc[mask, column_name] = -1
mask = df_standard.sentiment == 'positive'
column_name = 'sentiment'
df_standard.loc[mask, column_name] = 1

df_standard.head()

,review,sentiment
0,Very disappointed in the customer service. We ...,-1
1,I really wasn't thrilled with our meal here. T...,-1
2,STAY AWAY...\n\nWe've been 3 times over the pa...,-1
3,The food is good and the portions are large. ...,-1
4,I feel bad about giving this place such a meh ...,-1


In [81]:
# Train-Test Split + Stratify
df.drop(['rating'], inplace=True, axis=1)
df.head()

,review,sentiment
0,The service at Sushi Bar was great with an ext...,1
1,OMG Yelp let me down big time. I don't know h...,-1
2,Heard that this was the best sushi joint in Si...,1
3,Ebi fry salmon Aburizal ($17.90++) - 3 stars\n...,-1
4,Yes I also get the raving reviews now.\nWe dec...,1


In [82]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df,stratify=df['sentiment'], test_size=0.3) # We NEED more DATA In TEST

In [83]:
df_test

,review,sentiment
13,"Great food, quick service. Not the cheapest ea...",1
11,"Just go. Takes a bit of finding, but well wor...",1
19,Take away for this place is nice and fresh sea...,1
26,Been back 2 weeks and i have read much about t...,1
25,Randomly found this hidden sushi bar while wal...,1
1,OMG Yelp let me down big time. I don't know h...,-1
23,When the normal Chirashi bowl was served to me...,1
7,"Fresh fish. Generous portions. Fast, professio...",1
28,This is seriously one of my favourite places n...,1
9,Ars you looking for value for money sushi in a...,1


About one in ten in the test set, looks correctly stratified.

In [118]:
# df_train and df_standard are now the same format, we can concat and use it as a whole 
df_train_all = pd.concat([df_train, df_standard], axis=0)


## Naive Bayes Classifier

In [84]:
# Convert to dictionary format because apparently NLTK requires this
train_pos = [[row["review"], 1] for idx, row in df_train.iterrows() if row["sentiment"]==1]
train_neg = [[row["review"], -1] for idx, row in df_train.iterrows() if row["sentiment"]==-1]

test_pos = [[row["review"], 1] for idx, row in df_test.iterrows() if row["sentiment"]==1]
test_neg = [[row["review"], -1] for idx, row in df_test.iterrows() if row["sentiment"]==-1]

In [85]:
# need this part for max ent portion
testset = test_pos + test_neg
test_nolab = [t[0] for t in testset]
test_lab = [t[1] for t in testset]
test_tokenized = [[wt(x), c] for x,c in testset]
test_featureset = [(word_feats(d), c) for (d,c) in test_tokenized] 
test_nolab_tok = [t[0] for t in test_featureset]  # need to transform to predict

In [86]:
trainset = train_pos + train_neg
train_tokenized = [[wt(x), c] for x,c in trainset] # may need to introduce some pre-processing at this stage for better results

def word_feats(words):
    return dict([(word, True) for word in words])
train_featureset = [(word_feats(d), c) for (d,c) in train_tokenized] 

Prepare a combined training dataset of both scraped reviews and lecturer's standard set 

In [119]:
train_all_pos = [[row["review"], 1] for idx, row in df_train_all.iterrows() if row["sentiment"]==1]
train_all_neg = [[row["review"], -1] for idx, row in df_train_all.iterrows() if row["sentiment"]==-1]
trainset_all = train_all_pos + train_all_neg
train_all_tokenized = [[wt(x), c] for x,c in trainset_all] # may need to introduce some pre-processing at this stage for better results
train_all_featureset = [(word_feats(d), c) for (d,c) in train_all_tokenized] 

In [87]:
## Naive Bayes Rule using nltk
classifier_nb = NaiveBayesClassifier.train(train_featureset)
#print("Accuracy :" +str(accuracy(classifier_nb, test_featureset)))
classifier_nb.show_most_informative_features(10)

## Preparing the data first 
train_nolab = [t[0] for t in trainset]
train_lab = [t[1] for t in trainset]

# Preparing test set in same format
testset = test_pos + test_neg
test_nolab = [t[0] for t in testset]
test_lab = [t[1] for t in testset]

# Create your tf-idf function
vectorizer = TfidfVectorizer(max_df=0.7, min_df=3, use_idf=True) # modified this
train_vectors = vectorizer.fit_transform(train_nolab)
test_vectors = vectorizer.transform(test_nolab)

## train Naive Bayes Rule using sklearn
clf = MultinomialNB().fit(train_vectors, train_lab)

predNB = clf.predict(train_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(train_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1

Most Informative Features
                       , = None               -1 : 1      =      7.7 : 1.0
               expensive = True               -1 : 1      =      5.5 : 1.0
                    Nice = True               -1 : 1      =      5.5 : 1.0
                    kind = True               -1 : 1      =      5.5 : 1.0
                    full = True               -1 : 1      =      5.5 : 1.0
                      80 = True               -1 : 1      =      5.5 : 1.0
                     Not = True               -1 : 1      =      5.5 : 1.0
                    nice = True               -1 : 1      =      5.5 : 1.0
                    Just = True               -1 : 1      =      5.5 : 1.0
                     any = True               -1 : 1      =      5.5 : 1.0


pred,1,All
actuals,,
-1,3,3
1,21,21
All,24,24


May need some preprocessing. See '4' as an informative feature. Makes sense? 

Now test on the test set

In [88]:
predNB = clf.predict(test_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(test_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1


pred,1,All
actuals,,
-1,1,1
1,10,10
All,11,11


In [89]:
print (classification_report(pred,  test_lab))
print (accuracy_score(pred, test_lab))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11

0.9090909090909091


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Repeat Naive Bayes Classifier Using Combined Training Set

In [128]:
## Naive Bayes Rule using nltk
classifier_nb = NaiveBayesClassifier.train(train_all_featureset)
#print("Accuracy :" +str(accuracy(classifier_nb, test_featureset)))
classifier_nb.show_most_informative_features(10)

## Preparing the data first 
trainall_nolab = [t[0] for t in trainset_all]
trainall_lab = [t[1] for t in trainset_all]

# Create your tf-idf function
vectorizer = TfidfVectorizer(max_df=0.7, min_df=3, use_idf=True) # modified this
trainall_vectors = vectorizer.fit_transform(trainall_nolab)
test_vectors = vectorizer.transform(test_nolab)

## train Naive Bayes Rule using sklearn
clf = MultinomialNB().fit(trainall_vectors, trainall_lab)

predNB = clf.predict(trainall_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(trainall_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1

Most Informative Features
                Terrible = True               -1 : 1      =     49.5 : 1.0
              flavorless = True               -1 : 1      =     37.4 : 1.0
                Horrible = True               -1 : 1      =     36.5 : 1.0
               DELICIOUS = True                1 : -1     =     31.8 : 1.0
                     Meh = True               -1 : 1      =     31.0 : 1.0
                HORRIBLE = True               -1 : 1      =     29.1 : 1.0
            unacceptable = True               -1 : 1      =     28.0 : 1.0
                  rudely = True               -1 : 1      =     27.4 : 1.0
                   WORST = True               -1 : 1      =     24.6 : 1.0
               redeeming = True               -1 : 1      =     24.6 : 1.0


pred,-1,1,All
actuals,,,
-1,10063,951,11014
1,1295,8044,9339
All,11358,8995,20353


Now use full dataset-trained classifier on test set 

In [129]:
predNB = clf.predict(test_vectors)
pred = list(predNB)
cm1=pd.crosstab( pd.Series(test_lab), pd.Series(pred), rownames= ['actuals'], colnames=['pred'],margins=True)
cm1


pred,-1,1,All
actuals,,,
-1,1,0,1
1,3,7,10
All,4,7,11


In [130]:
print (classification_report(pred,  test_lab))
print (accuracy_score(pred, test_lab))

             precision    recall  f1-score   support

         -1       1.00      0.25      0.40         4
          1       0.70      1.00      0.82         7

avg / total       0.81      0.73      0.67        11

0.7272727272727273


## SVM Classifier

In [90]:
from sklearn.svm import SVC

# SVM Classifier from sklearn
def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    svm = SVC(C=10000.0, gamma='auto', kernel='rbf')
    svm.fit(X, y)
    return svm

classifier_svm = train_svm(train_vectors, train_lab)  # training the SVM model
predSVM = classifier_svm.predict(test_vectors) 
pred_svm = list(predSVM)

In [91]:
cm6=pd.crosstab( pd.Series(test_lab), pd.Series(pred_svm), rownames= ['actuals'], colnames=['pred'],margins=True)
cm6


pred,1,All
actuals,,
-1,1,1
1,10,10
All,11,11


In [92]:
print (classification_report(pred_svm,  test_lab))
print (accuracy_score(pred_svm, test_lab))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11

0.9090909090909091


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Repeat SVM Classifier using Entire Training Data Set

In [131]:
classifier_svm = train_svm(trainall_vectors, trainall_lab)  # training the SVM model
predSVM = classifier_svm.predict(test_vectors) 
pred_svm = list(predSVM)
cm6=pd.crosstab( pd.Series(test_lab), pd.Series(pred_svm), rownames= ['actuals'], colnames=['pred'],margins=True)
cm6


pred,-1,1,All
actuals,,,
-1,1,0,1
1,2,8,10
All,3,8,11


In [132]:
print (classification_report(pred_svm,  test_lab))
print (accuracy_score(pred_svm, test_lab))

             precision    recall  f1-score   support

         -1       1.00      0.33      0.50         3
          1       0.80      1.00      0.89         8

avg / total       0.85      0.82      0.78        11

0.8181818181818182


## Max Ent Classifier

In [93]:
classifier_me = MaxentClassifier.train(train_featureset, algorithm="IIS", max_iter=5)

  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.875
             2          -0.27948        0.875
             3          -0.23122        0.875
             4          -0.19607        0.917
         Final          -0.16931        0.958


In [94]:
pred_me = []
test_nolab_tok = [t[0] for t in test_featureset]  # need to transform to predict
for t in test_nolab_tok:
    pred_me.append(classifier_me.classify(t))

cm5=pd.crosstab( pd.Series(test_lab), pd.Series(pred_me), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm5)

pred      1  All
actuals         
-1        1    1
1        10   10
All      11   11


In [95]:
print (classification_report(pred_me,  test_lab))
print (accuracy_score(pred_me, test_lab))

             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11

0.9090909090909091


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Repeat Max Ent Classifier with Full Dataset

In [ ]:
classifier_me = MaxentClassifier.train(train_all_featureset, algorithm="IIS", max_iter=5)
pred_me = []
test_nolab_tok = [t[0] for t in test_featureset]  # need to transform to predict
for t in test_nolab_tok:
    pred_me.append(classifier_me.classify(t))

cm5=pd.crosstab( pd.Series(test_lab), pd.Series(pred_me), rownames= ['actuals'], colnames=['pred'],margins=True)
print (cm5)


  ==> Training (5 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.459
             2          -0.68658        0.472


In [ ]:
print (classification_report(pred_me,  test_lab))
print (accuracy_score(pred_me, test_lab))

## Use K Best Features 

In [96]:
# TODO update this to run on full dataset 

ch21 = SelectKBest(chi2, k=50) # TODO modify k according to number of features you want 
# Transform your training and testing datasets accordingly
train_Kbest = ch21.fit_transform(train_vectors, train_lab)
test_Kbest = ch21.transform(test_vectors)

## K Best SVM

In [97]:
# Train your SVM with the k best selected features
sv = train_svm(train_Kbest, train_lab)
predSVM= sv.predict(test_Kbest)
pred = list(predSVM)
cm8 = confusion_matrix(pred, test_lab)
print (cm8)
print (accuracy_score(pred, test_lab))
print (classification_report(pred,  test_lab))

[[ 0  0]
 [ 1 10]]
0.9090909090909091
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11



/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## K Best Naive Bayes

In [98]:
clf = MultinomialNB().fit(train_Kbest, train_lab)
predNB = clf.predict(test_Kbest)
pred = list(predNB)
cm9 = confusion_matrix(pred, test_lab)
print (cm9)
print (accuracy_score(pred, test_lab))
print (classification_report(pred,  test_lab))

# View the selected features
selected_features = list(np.array(vectorizer.get_feature_names())[ch21.get_support()])
print (selected_features)

[[ 0  0]
 [ 1 10]]
0.9090909090909091
             precision    recall  f1-score   support

         -1       0.00      0.00      0.00         0
          1       1.00      0.91      0.95        11

avg / total       1.00      0.91      0.95        11

['90', 'as', 'at', 'bar', 'be', 'before', 'better', 'chirashi', 'crowded', 'decent', 'dinner', 'dishes', 'fish', 'fresh', 'get', 'given', 'great', 'have', 'here', 'into', 'is', 'like', 'more', 'nice', 'not', 'on', 'or', 'out', 'people', 'pieces', 'place', 'portions', 'price', 'queue', 'rather', 'really', 'reservation', 'rice', 'sashimi', 'service', 'stars', 'than', 'they', 'understand', 'up', 'wait', 'were', 'which', 'would', 'you']


/Users/pierlim/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
